In [ ]:
import autograd.numpy as np
from autograd import grad, jacobian
from scipy.optimize import minimize
import copy
from model import create_ploy_basis_file, find_cq
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Times New Roman"

np.random.seed(1)

dimension = 12

# specify x
num_points = 5000
x = np.random.normal(0,1,(dimension,num_points))

# load basis file
create_ploy_basis_file(orders=[1,2,3,4], num_variable=dimension, path="./basis.txt")
bases = np.loadtxt('./basis.txt', dtype='str')


def f(x):
    
    xdot = np.zeros((dimension, x.shape[1]))
    
    lamda=((x[1]*x[7]-x[5]*x[3])+(x[5]*x[11]-x[9]*x[7])+(x[9]*x[3]-x[1]*x[11]))/((x[0]*x[4]+x[4]*x[8]+x[8]*x[0]+x[2]*x[6]+x[6]*x[10]+x[10]*x[2])-(x[0]**2+x[2]**2+x[4]**2+x[6]**2+x[8]**2+x[10]**2));

    xdot[0]=x[1];
    xdot[1]=lamda*(x[6]-x[10]);
    xdot[2]=x[3];
    xdot[3]=lamda*(x[8]-x[4]);
    xdot[4]=x[5];
    xdot[5]=lamda*(x[10]-x[2]);
    xdot[6]=x[7];
    xdot[7]=lamda*(x[0]-x[8]);
    xdot[8]=x[9];
    xdot[9]=lamda*(x[2]-x[6]);
    xdot[10]=x[11];
    xdot[11]=lamda*(x[4]-x[0]);

    xdot = xdot/np.linalg.norm(xdot, axis=0)[np.newaxis, :]
    return xdot


def get_data(num_points):
    x = np.random.normal(0,1,(dimension, num_points))
    x[11] = -(x[1]*(x[6] - x[10]) + (-x[4] + x[8])*x[3] + x[5]*(x[10] - x[2]) + x[7]*(x[0] - x[8]) + x[9]*(x[2] - x[6]))/(-x[0] + x[4]);
    return x

x = get_data(num_points)



In [ ]:
tol_cq = 1e-14
tol_dep = 1e-4
results = find_cq(f, x, bases, tol_cq=tol_cq, tol_dep=tol_dep, sparse_run=1, max_iter=0)

In [ ]:
s_cq = results['s_cq'][::-1]
n_cq_dep = np.sum(s_cq<tol_cq)
plt.plot(np.arange(s_cq.shape[0])+1,s_cq,marker="o")
plt.plot([1, s_cq.shape[0]+1],[tol_cq, tol_cq], ls="--",color="red")
#plt.plot([1, s_cq.shape[0]+1],[tol_cq*0.1, tol_cq*0.1], ls="--",color="green")
plt.text(5,1e-16,r"$M=$"+"{}".format(n_cq_dep),fontsize=15,color="red")
plt.yscale('log')
#plt.xticks(np.arange(s_cq.shape[0])+1);
plt.xlabel('index '+r"$i$", fontsize=15)
plt.ylabel('singulvar value '+r"$\sigma_i$", fontsize=15)

In [ ]:
S = results['s_cq_independent']
n_cq_indep = np.sum(np.mean(S, axis=0)>tol_dep)
plt.plot(np.arange(S.shape[1])+1,np.mean(S, axis=0),marker="o")
plt.plot([1, S.shape[1]],[tol_dep, tol_dep], ls="--",color="red")
plt.text(2.5,1e-1,r"$c=$"+"{}".format(n_cq_indep),fontsize=15,color="red")
plt.yscale('log')
plt.xticks(np.arange(S.shape[1])+1);
plt.xlabel('index '+r"$i$", fontsize=15)
plt.ylabel('singulvar value '+r"$s_i$", fontsize=15)

In [ ]:
plt.figure(figsize=(11,2.5))

plt.subplot(1,2,1)

s_cq = results['s_cq'][::-1]
plt.plot(np.arange(s_cq.shape[0])+1,s_cq,marker="o")
plt.plot([1, s_cq.shape[0]+1],[tol_cq, tol_cq], ls="--",color="red")
plt.text(2,1e-16,r"$M=$"+"{}".format(n_cq_dep)+" CQs (possibly dependent)",fontsize=15,color="red")
plt.yscale('log')
#plt.xticks(np.arange(s_cq.shape[0])+1);
#plt.xlabel('index '+r"$i$", fontsize=15)
plt.ylabel('singulvar value '+r"$\sigma_i$", fontsize=15)
plt.xlabel('index '+r"$i$", fontsize=15)

plt.subplot(1,2,2)
S = results['s_cq_independent']
plt.plot(np.arange(S.shape[1])+1,np.mean(S, axis=0),marker="o")
plt.plot([1, S.shape[1]],[tol_dep, tol_dep], ls="--",color="red")
plt.text(1.2,1e-3,r"$c=$"+"{}".format(n_cq_indep)+" CQs (independent)",fontsize=15,color="red")
plt.yscale('log')
plt.xticks(np.arange(S.shape[1])+1);
plt.xlabel('index '+r"$i$", fontsize=15)
plt.ylabel('singulvar value '+r"$s_i$", fontsize=15)

plt.suptitle('2D fluid: up to 4th order polynomials', fontsize=15, y=1.05)

plt.savefig('./fig/fluid_2D_4poly_sv.pdf', bbox_inches="tight")

In [ ]:
sols = results['sol_cq_independent']
plt.yticks(np.arange(n_cq_indep),np.arange(n_cq_indep)+1)

#plt.set_title(r'$\Theta^{(3)}$',fontsize=15)
plt.xlabel("basis",fontsize=12)
plt.ylabel("CQ",fontsize=12)
#plt.xticks(np.arange(11), ticks)
im = plt.imshow(sols, vmin=-1, vmax=1, aspect=4)
plt.colorbar(fraction=0.017, pad=0.04)
#plt.savefig('./fig/fluid_2D_CQ.pdf', bbox_inches="tight")